In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from train_utils import run_epoch, eval_rw
from blender_dataloader import BlenderDataloader
from rw_dataloader import RealWorldDataloader

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


In [2]:
iterates_to_combine = list(range(23, 31)) #[7, 8, 9, 10]
model_stem = '6.24'
run_id = '3qdtsae1'

In [3]:
#glob.glob(f"{BESPOKE_ROOT}/models/m_{model_stem}*")

In [4]:
m_paths = [f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch" for e in iterates_to_combine]
m_paths

['/media/beans/ssd/bespoke/models/m_6.24_e23.torch',
 '/media/beans/ssd/bespoke/models/m_6.24_e24.torch',
 '/media/beans/ssd/bespoke/models/m_6.24_e25.torch',
 '/media/beans/ssd/bespoke/models/m_6.24_e26.torch',
 '/media/beans/ssd/bespoke/models/m_6.24_e27.torch',
 '/media/beans/ssd/bespoke/models/m_6.24_e28.torch',
 '/media/beans/ssd/bespoke/models/m_6.24_e29.torch',
 '/media/beans/ssd/bespoke/models/m_6.24_e30.torch']

In [5]:
state_dicts = [torch.load(p) for p in m_paths]
state_dict_avg = state_dicts[0].copy() # placeholder for a sec

In [6]:
for k in state_dict_avg.keys():
    state_dict_avg[k] = sum([sd[k] for sd in state_dicts]) / len(state_dicts)

In [7]:
m = EffNet(model_arch="efficientnet_b3").to(device)
m.load_state_dict(state_dict_avg)

<All keys matched successfully>

In [8]:
#m.state_dict() # visibly inspect, weights should be in the neighborhood

In [9]:
BS = 64
dataloader = BlenderDataloader(path_stem="trn", bs=BS)

In [10]:
%%time
with torch.no_grad():
    run_epoch(dataloader, m, train=True, backwards=False, log_wandb=False, updates_per_epoch=256)

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


AttributeError: 'NoneType' object has no attribute 'param_groups'

In [11]:
torch.save(m.state_dict(), f'{BESPOKE_ROOT}/models/m_{model_stem}_avg.torch')

In [12]:
rw_dataloader = RealWorldDataloader()

(5400, 100, 640, 3) (5400, 5) (5400, 1)
(6000, 100, 640, 3) (6000, 5) (6000, 1)


In [13]:
import wandb
wandb.init(id=run_id, project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [14]:
eval_rw(rw_dataloader, m, wandb)

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
